# Online Retail Dataset: Create Database
## Imports

In [1]:
import datetime
import re
import sqlite3
from pathlib import Path

import polars as pl

## Check if DB already exists

In [2]:
data_dir = Path.cwd().parents[1] / "data"
db_path = data_dir / "online_retail.db"

In [3]:
assert not db_path.exists(), "DB already exists"
db_path.touch(mode=0o644)

## Check if Excel file exists

In [4]:
excel_file_path = db_path.with_suffix(".xlsx")
assert excel_file_path.exists(), "Excel data file doesn't exist"

## Create DB connection

In [5]:
con = sqlite3.connect(db_path)
cur = con.cursor()

## Create table to store the original data

In [6]:
sql = """
CREATE TABLE original_data (
    id INTEGER PRIMARY KEY,
    invoice_no TEXT NOT NULL,
    stock_code TEXT NOT NULL,
    description TEXT,
    quantity INTEGER NOT NULL,
    invoice_date TEXT NOT NULL,
    unit_price REAL NOT NULL,
    customer_id INTEGER,
    country TEXT
);
"""
cur = cur.execute(sql)

In [7]:
# Check if table was created
res = cur.execute("SELECT name FROM sqlite_master;")
assert res.fetchone() == ("original_data",), "failed to create table"

## Read original data file

In [8]:
# Read every column as a set of strings
cols = [
    "InvoiceNo",
    "StockCode",
    "Description",
    "Quantity",
    "InvoiceDate",
    "UnitPrice",
    "CustomerID",
    "Country",
]
df = pl.read_excel(
    excel_file_path,
    schema_overrides={col: pl.String for col in cols},
)

## Missing values

In [9]:
df.null_count()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
u32,u32,u32,u32,u32,u32,u32,u32
0,0,1454,0,0,0,135080,0


## Checking and fixing the data
### `InvoiceNo`

`InvoiceNo` cannot be converted into an integer column. Some values are
not numbers:

In [10]:
tmp_df = df.filter(pl.col("InvoiceNo").str.find(r"^\d+$").is_null())
tmp_df.head()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,str,str,str,str,str
"""C536379""","""D""","""Discount""","""-1""","""2010-12-01 09:41:00""","""27.5""","""14527""","""United Kingdom"""
"""C536383""","""35004C""","""SET OF 3 COLOURED FLYING DUCK…","""-1""","""2010-12-01 09:49:00""","""4.65""","""15311""","""United Kingdom"""
"""C536391""","""22556""","""PLASTERS IN TIN CIRCUS PARADE ""","""-12""","""2010-12-01 10:24:00""","""1.65""","""17548""","""United Kingdom"""
"""C536391""","""21984""","""PACK OF 12 PINK PAISLEY TISSUE…","""-24""","""2010-12-01 10:24:00""","""0.29""","""17548""","""United Kingdom"""
"""C536391""","""21983""","""PACK OF 12 BLUE PAISLEY TISSUE…","""-24""","""2010-12-01 10:24:00""","""0.29""","""17548""","""United Kingdom"""


In [11]:
tmp_df.height

9291

In [12]:
new_col = df.get_column("InvoiceNo").str.strip_chars()
assert new_col.str.len_chars().gt(0).all(), "found empty string"
df = df.replace_column(df.get_column_index("InvoiceNo"), new_col)

### `StockCode`

`StockCode` will also remain as a string column. The corresponding values
contain numbers and letters. Making sure all of these strings are
non-empty:

In [13]:
new_col = df.get_column("StockCode").str.strip_chars()
assert new_col.str.len_chars().gt(0).all(), "found empty string"
df = df.replace_column(df.get_column_index("StockCode"), new_col)

### `Description`

`Description` is obviously a string column. Then I'll just normalize the
corresponding data.

In [14]:
# Normalization
new_col = df.get_column("Description").str.strip_chars()
new_col = new_col.str.strip_chars_end(".")
new_col = new_col.str.replace_all(r" {2,}", " ")
new_col = new_col.str.to_uppercase()

In [15]:
# Non-null values are non-empty strings
assert new_col.drop_nulls().str.len_chars().gt(0).all(), "found empty string"

In [16]:
df = df.replace_column(df.get_column_index("Description"), new_col)

### `Quantity`

`Quantity` should be an integer column. Its values can be positive or
negative. First, I'll check that this is actually the case:

In [17]:
new_col = df.get_column("Quantity").str.strip_chars()
assert new_col.str.contains(r"^-?\d+$").all(), "found non-integer value"

Next, we convert `Quantity` into an integer column:

In [18]:
new_col = new_col.str.to_integer()
df = df.replace_column(df.get_column_index("Quantity"), new_col)

### `InvoiceDate`

`InvoiceDate` should contain a date and a time. We begin by checking if
this is true:

In [19]:
new_col = df.get_column("InvoiceDate").str.strip_chars()
assert new_col.str.contains(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$").all(), "found invalid date"

Then we can convert `InvoiceDate` to a more appropriate type:

In [20]:
new_col = new_col.str.to_datetime("%Y-%m-%d %H:%M:%S")
df = df.replace_column(df.get_column_index("InvoiceDate"), new_col)

### `UnitPrice`

`UnitPrice` is supposed to be a column with real numbers. First, we check
if this is the case:

In [21]:
new_col = df.get_column("UnitPrice").str.strip_chars()
assert new_col.str.contains(r"^-?\d+(\.\d+)?$").all(), "found invalid price"

So it's safe to convert these prices into real numbers:

In [22]:
new_col = new_col.cast(pl.Float64)
df = df.replace_column(df.get_column_index("UnitPrice"), new_col)

### `CustomerID`

`CustomerID` has missing values. However, the values that are *not*
missing should be integers. Let's confirm that:

In [23]:
new_col = df.get_column("CustomerID").str.strip_chars()
assert new_col.drop_nulls().str.contains(r"^\d+$").all(), "found non-integer ID"

Next, we convert `CustomerID` into an integer column:

In [24]:
new_col = new_col.str.to_integer()
df = df.replace_column(df.get_column_index("CustomerID"), new_col)

### `Country`

`Country` is definitely a string column. So it's not necessary to change its
type. However, there's a change I want to make. As it is, this column has no
missing value. But that's not really true. Some values are "Unspecified":

In [25]:
new_col = df.get_column("Country").str.strip_chars()
tmp = new_col.filter(new_col.eq("Unspecified"))
tmp.head()

Country
str
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""
"""Unspecified"""


In [26]:
tmp.len()

446

I want to make it clear that these are missing values. This can be done as
follows:

In [27]:
new_col = new_col.set(new_col.eq("Unspecified"), None)
assert new_col.null_count() == tmp.len(), "wrong transformation"

Finally, we can replace the `Country` column:

In [28]:
assert new_col.drop_nulls().str.len_chars().gt(0).all(), "found empty string"
df = df.replace_column(df.get_column_index("Country"), new_col)

### Rename DataFrame columns

In [29]:
def pascal_to_snake(s: str) -> str:
    """Convert from Pascal case to snake case."""
    return re.sub(r"(?<!^)(?=[A-Z])", "_", s).lower()

In [30]:
# New column names to match the SQL table
snake_cols = [pascal_to_snake(col) for col in cols]
snake_cols[cols.index("CustomerID")] = "customer_id"

In [31]:
df = df.rename({old_name: new_name for old_name, new_name in zip(cols, snake_cols)})
assert df.columns == snake_cols, "failed to rename columns"

In [32]:
df.head()

invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
str,str,str,i64,datetime[μs],f64,i64,str
"""536365""","""85123A""","""WHITE HANGING HEART T-LIGHT HO…",6,2010-12-01 08:26:00,2.55,17850,"""United Kingdom"""
"""536365""","""71053""","""WHITE METAL LANTERN""",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom"""
"""536365""","""84406B""","""CREAM CUPID HEARTS COAT HANGER""",8,2010-12-01 08:26:00,2.75,17850,"""United Kingdom"""
"""536365""","""84029G""","""KNITTED UNION FLAG HOT WATER B…",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom"""
"""536365""","""84029E""","""RED WOOLLY HOTTIE WHITE HEART""",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom"""


## Adapter for `datetime.datetime`

In [33]:
def adapt_datetime_iso(val: datetime.datetime) -> str:
    """Adapt datetime.datetime to timezone-naive ISO 8601 date."""
    return val.isoformat(sep=" ")


sqlite3.register_adapter(datetime.datetime, adapt_datetime_iso)

## Insert data into DB

Finally, we can insert the data into the database. Probably, this is not
the most efficient way to do so. But we're going to do this only once. So
possible inefficiency is not a big deal.

In [34]:
# Build SQL statement
col_names = ", ".join(snake_cols)
placeholders = ", ".join(f":{col}" for col in snake_cols)
sql = f"INSERT INTO original_data ({col_names}) VALUES ({placeholders});"

In [35]:
# Execute statement
cur.executemany(sql, df.to_dicts())
con.commit()

In [36]:
res = cur.execute("SELECT COUNT(*) FROM original_data;")
assert res.fetchone() == (df.height,), "failed to insert all rows"

## Close DB connection

In [37]:
con.commit()  # Just to be sure
con.close()